In [ ]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

In [ ]:
!pip install pillow

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
import json

from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
subscription_key = "afaedacab74e46f5a749bbc928a8e89b"
endpoint = "https://20210508-donakuma.cognitiveservices.azure.com/"

In [ ]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

## Read API を呼び出す

In [ ]:
print("===== Read File - remote =====")

remote_image_handw_text_url = "https://raw.githubusercontent.com/MicrosoftDocs/azure-docs/master/articles/cognitive-services/Computer-vision/Images/readsample.jpg"

recognize_handw_results = computervision_client.read(remote_image_handw_text_url,  raw=True)

## 読み取りの結果を取得する

In [ ]:
# Get the operation location (URL with an ID at the end) from the response
operation_location_remote = recognize_handw_results.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = operation_location_remote.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    get_handw_text_results = computervision_client.get_read_result(operation_id)
    if get_handw_text_results.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if get_handw_text_results.status == OperationStatusCodes.succeeded:
    for text_result in get_handw_text_results.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()

## ローカルファイルに対応させる

### 手書き

In [95]:
print("===== Read File - local =====")
# Get image of handwriting
# local_image_handwritten_path = os.path.join (images_folder, "MyProgram\readsample.jpg")
local_image_handwritten_path = "img\\20210513_230148000_iOS.jpg"
# Open the image
local_image_handwritten = open(local_image_handwritten_path, "rb")

# Call API with image and raw response (allows you to get the operation location)
recognize_handwriting_results = computervision_client.read_in_stream(local_image_handwritten, raw=True)
# Get the operation location (URL with ID as last appendage)
operation_location_local = recognize_handwriting_results.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id_local = operation_location_local.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    recognize_handwriting_result = computervision_client.get_read_result(operation_id_local)
    if recognize_handwriting_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

# Print results, line by line

img = Image.open(local_image_handwritten_path)
draw = ImageDraw.Draw(img)
font = ImageFont.truetype(font='meiryo.ttc', size=10)

if recognize_handwriting_result.status == OperationStatusCodes.succeeded:
    for text_result in recognize_handwriting_result.analyze_result.read_results:
        for line in text_result.lines:

            print(line.text)
            print(line.bounding_box)
#             img_draw = line.bounding_box
#             draw.rectangle([(img_draw[0], img_draw[1]),
#                             (img_draw[4], img_draw[5])],
#                             fill=None,
#                             outline='yellow',
#                             width=5)
#             text_w, text_y = draw.textsize(line.text, font=font)
#             draw.rectangle([(img_draw[0], img_draw[1]),
#                             (img_draw[0]+text_w, img_draw[1]+text_y)],            
#                             fill='yellow', width=5)
#             draw.text((img_draw[0], img_draw[1]), line.text, fill='black', font=font)

# img

===== Read File - local =====
Waiting for result...
FamilyMart
[1751.0, 544.0, 2981.0, 477.0, 2994.0, 711.0, 1764.0, 774.0]
相模大野八丁目店
[1416.0, 784.0, 2207.0, 765.0, 2208.0, 865.0, 1416.0, 882.0]
神奈川県相模原市南区相模大野
[1402.0, 878.0, 2802.0, 815.0, 2807.0, 925.0, 1402.0, 981.0]
八丁目9-10
[1412.0, 970.0, 2092.0, 956.0, 2095.0, 1048.0, 1412.0, 1068.0]
電話:042-767-2032
[1424.0, 1062.0, 2307.0, 1037.0, 2309.0, 1134.0, 1424.0, 1166.0]
2021年 5月13日(木) 20:30
[1411.0, 1253.0, 2666.0, 1207.0, 2670.0, 1309.0, 1415.0, 1343.0]
レジ 1-0789
[1409.0, 1346.0, 1916.0, 1336.0, 1919.0, 1429.0, 1410.0, 1439.0]
責No. 012
[2609.0, 1306.0, 3047.0, 1276.0, 3048.0, 1373.0, 2612.0, 1406.0]
野菜ましラーメンサラダ
[1407.0, 1529.0, 2512.0, 1509.0, 2515.0, 1610.0, 1410.0, 1644.0]
領
[1855.0, 1429.0, 1977.0, 1429.0, 1980.0, 1525.0, 1859.0, 1520.0]
収
[2155.0, 1426.0, 2263.0, 1416.0, 2266.0, 1513.0, 2158.0, 1520.0]
証
[2451.0, 1420.0, 2561.0, 1409.0, 2571.0, 1497.0, 2458.0, 1506.0]
¥430軽
[2722.0, 1494.0, 3049.0, 1476.0, 3054.0, 1573.0, 2727.0, 15

In [108]:
# print(text_result.lines[0])
t1 = recognize_handwriting_result.analyze_result
# print(t1)
print(t1.read_results[0].lines[0])
# print(recognize_handwriting_result.analyze_result.read_results[0])

{'additional_properties': {}, 'language': None, 'bounding_box': [1751.0, 544.0, 2981.0, 477.0, 2994.0, 711.0, 1764.0, 774.0], 'appearance': <azure.cognitiveservices.vision.computervision.models._models_py3.Appearance object at 0x00000113FC6B7730>, 'text': 'FamilyMart', 'words': [<azure.cognitiveservices.vision.computervision.models._models_py3.Word object at 0x00000113FC6B7760>]}


In [ ]:
img = Image.open(local_image_handwritten_path)
draw = ImageDraw.Draw(img)
font = ImageFont.truetype(font='meiryo.ttc', size=50)



In [ ]:
for bounding_list in bounding_lists:
    img_draw = bounding_list
    draw.polygon((
        (img_draw[0], img_draw[1]),
        (img_draw[2], img_draw[3]),
        (img_draw[4], img_draw[5]),
        (img_draw[6], img_draw[7])),
        fill=None,
        outline='yellow',
    )

img

In [ ]:
for bounding_list in bounding_lists:
    img_draw = bounding_list
    draw.rectangle((
        (img_draw[0], img_draw[1]),
        (img_draw[4], img_draw[5])),
        fill=None,
        outline='yellow',
        width=5
    )

img

In [ ]:
bounding_lists

In [ ]:
img_draw[0]

In [ ]:
print(line.appearance.style)

In [ ]:
img_draw = bounding_list[0]['bounding']
img_draw

In [ ]:
bounding_list[0]['bounding'][0]

In [ ]:
from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
img = Image.open("wakeupcat.jpg")
draw = ImageDraw.Draw(img)
draw.polygon(((img_draw[0], img_draw[1]),(img_draw[2], img_draw[3]),(img_draw[4], img_draw[5]),(img_draw[6], img_draw[7])), outline='red')
img